In [ ]:
import numpy as np
from numpy.linalg import inv, det, matrix_rank
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal

def g(x):
    return -np.power(np.sin(6*np.pi*x), 2) + 6 * np.power(x, 2) - 5 * np.power(x, 4) + 3/2


def gauss_kernel(x: float, y: float, gamma: float=1.) -> float:
    return np.exp(-gamma*np.power((x-y), 2))

def period_kernel(x, z, var=0.1, p=0.2, lp=5, lse=0.5):
    """
    :param sig2: variance
    :param p: period
    :param lp: length parameter of periodic kernel
    :param lse: length parameter of squared exponential kernel 
    """
    periodic_kernel = var * np.exp(- 2 * np.power(np.sin(np.pi*np.abs(x - z)/p), 2)/ np.power(lp, 2))
    squared_exponential_kernel = np.exp(-np.power(x-z, 2)/2*np.power(lse, 2))
    return periodic_kernel * squared_exponential_kernel

In [ ]:
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal

l=101
X_test = np.array([(i-1)/(l-1) for i in range(1,l+1)])
X_train = np.array([0,0.25,0.5])
Y_train = np.array([1.46, 0.93, 2.76])
W = np.array([1/(2*l-2) if i in [1,l] else 1/(l-1) for i in range(1,l+1)]).reshape(l,1)
q= np.array([2])

def K(X,Z):
    return np.array([[period_kernel(x,z) for z in Z] for x in X])

# Compute f*|f
var = 0.01
mean_fstar = K(X_train, X_test).T @ inv(K(X_train,X_train) + var*np.eye(3)) @ Y_train
var_fstar = K(X_test, X_test) - K(X_train, X_test).T @ inv(K(X_train, X_train) + var*np.eye(3)) @ K(X_train, X_test)

# Compute variance of (q_hat, f*) | f
"""
Sig_joint = np.zeros([102,102])
Sig_joint[0,0] = W.T @ var_fstar.T @ W
Sig_joint[1:,0] = (var_fstar @ W).reshape(101)
Sig_joint[0,1:] = W.T @ var_fstar
Sig_joint[1:,1:] = var_fstar
"""


# Compute f* | (q, f)
mu_final = var_fstar @ W @ inv(W.T @ var_fstar.T @ W) @ q
Sig_final = var_fstar - var_fstar @ W @ inv(W.T @ var_fstar.T @ W) @ (var_fstar @ W).T

# Sample
samples = multivariate_normal(mu_final, Sig_final, 3)
plt.plot(X_test, samples[0,:], c="blue", label="Prediction")
#plt.scatter(X_test, samples[1,:])
#plt.scatter(X_test, samples[2,:])
plt.scatter(X_train, Y_train, c="red", label="Training points")

# Plot true function
ys = [g(x) for x in X_test]
plt.plot(X_test, ys, c="black", ls="--", label="Latent function")
plt.legend()